<a href="https://colab.research.google.com/github/Raphael181/Raphael181/blob/main/TensorFlow_Sentiment_Analysis_with_RNN's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Sentiment Analysis to determine if movie reviews are either positive or negative
 #imports
 %tensorflow_version 2.x
 from keras.datasets import imdb
 from tensorflow import keras
 from keras.preprocessing import sequence
 import tensorflow as tf
 import os
 import numpy as np

In [ ]:
#Define the vobulary size
vocab_size = 88584
#max length of a review
maxlen = 250
#batch size
batch_size  = 64
#load dataset
(training , training_label) , (testing , testing_label)  = imdb.load_data(num_words  = vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
#Preprocesing: We cannot pass different length data into our model so we have to make all their lengths the same
train_data = sequence.pad_sequences(training , maxlen)
test_data = sequence.pad_sequences(testing , maxlen)

In [ ]:
#creating the model
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size , 32),
                tf.keras.layers.LSTM(32),
                tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#compile and train the model
model.compile(optimizer='rmsprop' , loss='binary_crossentropy',metrics = ['accuracy'])

history = model.fit(train_data, training_label , validation_split=0.2, epochs=5)

Epoch 1/5
625/625 [==============================] - 54s 83ms/step - loss: 0.5415 - accuracy: 0.7090 - val_loss: 0.2995 - val_accuracy: 0.8810
Epoch 2/5
625/625 [==============================] - 52s 83ms/step - loss: 0.2388 - accuracy: 0.9122 - val_loss: 0.2834 - val_accuracy: 0.8888
Epoch 3/5
625/625 [==============================] - 51s 82ms/step - loss: 0.1743 - accuracy: 0.9393 - val_loss: 0.3157 - val_accuracy: 0.8900
Epoch 4/5
625/625 [==============================] - 52s 83ms/step - loss: 0.1483 - accuracy: 0.9489 - val_loss: 0.2847 - val_accuracy: 0.8930
Epoch 5/5
625/625 [==============================] - 51s 81ms/step - loss: 0.1170 - accuracy: 0.9613 - val_loss: 0.2898 - val_accuracy: 0.8896


In [ ]:
#test the model 
result = model.evaluate(test_data , testing_label)
print(result)

782/782 [==============================] - 15s 19ms/step - loss: 0.3434 - accuracy: 0.8670
[0.3434389531612396, 0.8670399785041809]


In [ ]:
word_index = imdb.get_word_index()
#a function that encodes any words into proper preprocessed integers
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens =[word_index[word] if word in word_index else 0  for word in tokens]
  return sequence.pad_sequences([tokens] , maxlen=maxlen)[0]

text  = "that movie was amazing, so splendid"
encoded = encode_text(text)
print(encoded)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
#Decoding function
reverse_word_index = {value: key for (key,value) in word_index.items()}

def decode_integers(integers):
  Pad = 0
  text = ''
  for num in integers:
    if num != Pad:
      text += reverse_word_index[num] + " "
    
  return text[:-1]

print(decode_integers(encoded))

that movie was amazing so splendid


In [ ]:
#writing a function to predict

def prediction(text):
  #preprocessing statement to encode text
  encoded_text = encode_text(text)
  #create a blank numpy array
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_rev = "That was such as good movie"
prediction(positive_rev)

negative_rev = "That was such an awful movie"
prediction(negative_rev)

[0.37797326]
[0.26012635]


In [ ]:
#Example 2: Character Generator

path_to_file  = tf.keras.utils.get_file('shakespeare.txt' , 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
#open and read the file
text = open(path_to_file ,'rb').read().decode(encoding = 'utf-8')
#length of text is the number of characters in it
print(f"Length of text is {len(text)} characters" )

Length of text is 1115394 characters


In [ ]:
#preprocessing and encoding words
vocab = sorted(set(text))
#creating a mapping for unique characters to indices
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

#A function that changes text to int
def text_to_int(text):
  return np.array([char2idx[c] for c in text])

textasint = text_to_int(text)

In [ ]:
print("Text: " , text[:15])
print("Encoded:" , text_to_int(text[:15]))

Text:  First Citizen:

Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0]


In [ ]:
#Decoding/Converting numeric values to text:
def int_to_text(integer):
  try:
    integer = integer.numpy()
  except:
    pass
  return ''.join(idx2char[integer])
  
print(int_to_text(textasint[:15]))

First Citizen:



In [ ]:
#creating training examples
seq_length = 100
amountperepoch = len(text)//(seq_length + 1)

#create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(textasint)

In [ ]:
#turn the stream of characters into batches of desired length
sequences = char_dataset.batch(batch_size=)